In [1]:
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Any, TypedDict
from dotenv import load_dotenv
from langchain.schema import HumanMessage, SystemMessage
from langgraph.checkpoint.memory import InMemorySaver
load_dotenv()

True

In [2]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash") # Specify the model here


In [5]:
class JokeState(TypedDict):
    topic: str
    joke : str
    explanation : str
    

In [ ]:
def generate_joke(state: JokeState):
    prompt = f'generate a joke for the following topic {state["topic"]}'
    response = model.invoke(prompt).content
    return {"joke": response}

In [7]:
def generate_explanation(state: JokeState):
    prompt = f'write an explanation for the following joke {state["joke"]}'
    response = model.invoke(prompt).content
    return {"explanation": response}

In [9]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)

graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START , 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow  = graph.compile(checkpointer=checkpointer)

In [19]:
config1 = {'configurable': {'thread_id': 1}}

In [22]:
input = {"topic": "unemployment in the cs department"}
workflow.invoke(input,config=config1)

{'topic': 'unemployment in the cs department',
 'joke': "Why are CS grads so good at dealing with unemployment?\n\nBecause they're experts at 'resource management' and have already optimized the data structure for the change they find under their couch cushions.",
 'explanation': 'This joke plays on the stereotype that computer science (CS) graduates are highly analytical and skilled in specific technical domains, and humorously applies those skills to a very mundane and unfortunate real-world situation: unemployment and financial hardship.\n\nHere\'s the breakdown:\n\n1.  **"Why are CS grads so good at dealing with unemployment?"**\n    *   This sets up the premise, implying that unemployment is a challenge, and CS grads have a unique way of tackling it.\n\n2.  **"Because they\'re experts at \'resource management\'..."**\n    *   **In CS terms:** "Resource management" is a core concept in computer science. It refers to the efficient allocation and deallocation of system resources like

In [25]:
list(workflow.get_state_history(config=config1))

[StateSnapshot(values={'topic': 'unemployment in the cs department', 'joke': "Why are CS grads so good at dealing with unemployment?\n\nBecause they're experts at 'resource management' and have already optimized the data structure for the change they find under their couch cushions.", 'explanation': 'This joke plays on the stereotype that computer science (CS) graduates are highly analytical and skilled in specific technical domains, and humorously applies those skills to a very mundane and unfortunate real-world situation: unemployment and financial hardship.\n\nHere\'s the breakdown:\n\n1.  **"Why are CS grads so good at dealing with unemployment?"**\n    *   This sets up the premise, implying that unemployment is a challenge, and CS grads have a unique way of tackling it.\n\n2.  **"Because they\'re experts at \'resource management\'..."**\n    *   **In CS terms:** "Resource management" is a core concept in computer science. It refers to the efficient allocation and deallocation of s